In [1]:
%pwd

'c:\\Users\\keert\\OneDrive\\Documents\\JHUSenior\\ComputerVision\\hw1\\project'

In [31]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import imageio

In [44]:
dataset_type = "UCSD"

if dataset_type == "UCSD":
    dataset = "Data/Train016/"
    arr = os.listdir(dataset)
    arr.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
    print(len(arr))

    # first frame
    first_frame = cv2.imread(dataset + arr[0])
    print(first_frame.shape)

150
(240, 360, 3)


In [45]:
# generate initial corners of detected object
# set limit, minimum distance in pixels and quality of object corner to be tracked
parameters_shitomasi = dict(maxCorners=100, qualityLevel=0.3, minDistance=7)

# convert to grayscale
frame_gray_init = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

# Use Shi-Tomasi to detect object corners / edges from initial frame
edges = cv2.goodFeaturesToTrack(frame_gray_init, mask = None, **parameters_shitomasi)

# create a black canvas the size of the initial frame
canvas = np.zeros_like(first_frame)

# set min size of tracked object, e.g. 15x15px
parameter_lucas_kanade = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [47]:
image_list = []
# image_list.append(first_frame)
for i in range(1, len(arr)):
    # get next frame
    if dataset_type == "UCSD":
        frame = cv2.imread(dataset + arr[i])
    # prepare grayscale image
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # update object corners by comparing with found edges in initial frame
    update_edges, status, errors = cv2.calcOpticalFlowPyrLK(frame_gray_init, frame_gray, edges, None,
                                                         **parameter_lucas_kanade)
    # only update edges if algorithm successfully tracked
    new_edges = update_edges[status == 1]
    # to calculate directional flow we need to compare with previous position
    old_edges = edges[status == 1]

    canvas[canvas > 10] -= 10
    for i, (new, old) in enumerate(zip(new_edges, old_edges)):
        a, b = new.ravel()
        c, d = old.ravel()

        # draw line between old and new corner point with random colour
        mask = cv2.line(canvas, (int(a), int(b)), (int(c), int(d)), [254, 254, 254], 2)
        # draw circle around new position
        frame = cv2.circle(frame, (int(a), int(b)), 5, [254, 254, 254], -1)

    # result = cv2.add(frame, mask)
    result = mask.copy()
    image_list.append(result)
    cv2.imshow('Optical Flow (sparse)', result)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    # overwrite initial frame with current before restarting the loop
    frame_gray_init = frame_gray.copy()
    # update to new edges before restarting the loop
    edges = new_edges.reshape(-1, 1, 2)

cv2.destroyAllWindows()
print(len(image_list))
imageio.mimsave("OpticalFlowTrack.gif", image_list, fps=30)

150


In [14]:
cap = cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)
# 240, 360

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (640,480))

while(True):
    ret, frame = cap.read()
    out.write(frame)
    cv2.imshow('frame', frame)
    c = cv2.waitKey(1)
    if c & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [48]:
print(image_list[0].shape)

(240, 360, 3)


In [51]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (360,240))

for i in range(len(image_list)):
    out.write(image_list[i])
    c = cv2.waitKey(1)
    if c & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()